#### <span style="color:#grey"> __Formation South Green 2022 - Structural Variants Detection by using short and long reads__ </span>

# <span style="color:#006E7F">  <center> __DAY 2 : How to analyze mapping results ?__ </center> </span>

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)

***

# <span style="color: #006E7F">Table of contents</span>
<a class="anchor" id="home"></a>

[I - Get some basic mapping stats with samtools flagstat](#mappingstats)

* [Run samtools flagstat](#flagstat)
* [Samtools flagstat output](#flagstatoutput)
* [Merge individual flagstat files into an unique file with python code](#multiflagstat) 
* [Plot mapping ratio per sample](#ratioplot)
* [EXERCICE : DO THE SAME MANIP WITH MINIMAP2 RESULTS](#minimap)

[II - Get some basic stats from vcf files](#statvcf) 
* [Count the number of variants with `bcftools stat`](#bcftools)
* [Generating statistics from a VCF to determining how to set filters on it](#vcffilters)
* [Generating density plot QUAL & DEPTH](#vcfplot) 

</span>

***



## <span style="color:#006E7F">__I - Get some basic mapping stats with samtools flagstat__ <a class="anchor" id="mappingstats"></span>  

### <span style="color: #4CACBC;"> First go into the directory that contains all the bam files and list the content of the directory</span>  

As we will mainly launch python code to analyse mapping results, the kernel of this jupyter book is Python3. So we are going to add `%` to execute some specific linux commands or `%%bash`  to execute any linux command.

In [ ]:
%cd /home/jovyan/work/MAPPING-ILL/
%ls

### <span style="color: #4CACBC;">Grouping the flagstat files previosly created</span>  

* Create the subdirectory FLAGSTAT into the directory MAPPING_ILL
* copy the flagstat files previously generated into this new directry

In [ ]:
%%bash


#### <span style="color: #4CACBC;">Check that the flagstat files have been correctly copied</span>

### <span style="color: #4CACBC;">Let's look the content of one flagstat file <a class="anchor" id="flagstatoutput"></span> 

### <span style="color: #4CACBC;">Merge individual flagstat files into an unique file with python code <a class="anchor" id="multiflagstat"></span> 

In [ ]:
# IMPORT PYTHON PACKAGE USED BY THE CODE
import os
import pandas as pd

# VARIABLE INITIALIZATION

## NAME OF THE DIRECTORY THAT CONTAINS FLAGSTAT FILES
flagstat_dir = "/home/jovyan/work/MAPPING-ILL/FLAGSTAT" #PUT THE DIRECTORY NAME THAT CONTAINS FLAGSTAT FILES 

## NAME OF THE FILE THAT WILL CONTAIN ALL THE FLAGSTAT RESULTATS
stat_file = f"{flagstat_dir}/all_stat.csv"

# PRINT THE CONTENT OF 2 PREVIOUS VARIABLES INITIALIZED
print("DIRECTORY : ",flagstat_dir)
print("FINAL STAT FILE : ",stat_file)


In [ ]:
# OPEN THE FINAL FILE IN WHICH WE PRINT SOME STATS EXTRACTED FROM EACH INDIVIDUAL FILE GENERATED BY SAMTOOLS FLAGSTAT
with open(stat_file, 'w') as stat: 
    # WRITE A HEADER LINE IN OUR STAT FILE
    header_line = "sample,mapped,paired,unmapped"
    stat.write(header_line)
    
    # READING EACH FILE OF THE FLGSTAT DIRECTORY
    for file in os.listdir(flagstat_dir):
        # If flagstat is in name of file
        filen = flagstat_dir + "/" + file
        if "flagstat" in file:
            # Extract sample name and save into a new variable newLine 
            new_line = f"\n{file.split('.')[0]},"
            # OPEN AND READS FLAGSTAT FILE
            with open(filen, "r") as flagstat:
                # read file line by line
                for line in flagstat:
                    # remove the line skipper at the endo of the line
                    line = line.rstrip()              
                    # Keep only line mapped, paired or singleton word
                    if 'mapped (' in line or 'paired (' in line or 'singleton' in line:
                        # get percentage value and save it into the varaible called perc
                        perc = f"{line.split('(')[1].split('%')[0]}"
                        new_line += f"{perc},"
                # WRITE THE LINE ONCE THE FLAGSTAT FILE COMPLETELY READ
                stat.write(new_line.strip(","))

### <span style="color: #4CACBC;">Display the content of the final stat file  <a class="anchor" id="statfile"></span> 

### <span style="color: #4CACBC;">Plot mapping ratio per sample <a class="anchor" id="ratioplot"></a></span> 

#### Load csv file into a panda datafrale


In [ ]:
df_bam_stat = pd.read_csv(stat_file, index_col=False, sep=",")
df_bam_stat

#### Basic stats

In [ ]:
# Je n'affiche que les valeurs de la colonne "mapped"
print(df_bam_stat['mapped'])

In [ ]:
# J affiche la moyenne, min et max de cette colonne
minimum = df_bam_stat["mapped"].min()
maximun = df_bam_stat["mapped"].max()
mean_flag = df_bam_stat["mapped"].mean()

print("\n######## BASIC STATS\n MAPPED")       
print(f"\t%min : {minimum}\t %max : {maximun}\t %mean : {mean_flag}")


#### Sort by clone name

In [ ]:
df_bam_stat_sorted=df_bam_stat.sort_values(by=['sample'])
df_bam_stat_sorted

#### Your first plot with python

In [ ]:
# Plot with seaborn
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (15,8))
sns.scatterplot(x="sample",y="paired", data=df_bam_stat_sorted)

In [ ]:
# Plot with seaborn
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (15,8))
ax=sns.scatterplot(x="sample",y="value", hue='variable', data=pd.melt(df_bam_stat_sorted, 'sample'))
ax.set_title("PUT YOUR TITLE")
ax.set_xlabel("PUT YOUR X-AXIS NAME")
ax.set_ylabel("PUT YOUY Y-AXIS NAME")

### <span style="color: #4CACBC;"> EXERCICE : DO THE SAME THING WITH MINIMAP2 RESULTS... or not<a class="anchor" id="minimap"></span> 

In [ ]:
%%bash
cd /home/jovyan/work/
wget https://itrop.ird.fr/sv-training/VCF_REAL.tar.gz
tar zxvf VCF_REAL.tar.gz
rm VCF_REAL.tar.gz

### <span style="color: #4CACBC;">First go into the directory that contains vcf file  </span> 

* List the content of this directory


In [ ]:
cd /home/jovyan/work/VCF_REAL/

In [ ]:
ls

### <span style="color: #4CACBC;">Count the number of variants with `bcftools stat`<a class="anchor" id="bcftools"></a></span> 

* run the bcftools stats on the vcf file and save the result into the file `rice-CHR6.200000.vcf.stat`
* check that the file have been correctly created and display the 35 first lines of this file
* How many samples were used for this SNP analysis ?
* How many SNPs were detected ?

In [ ]:
%%bash
bcftools stats rice-CHR6.200000.vcf.gz >  rice-CHR6.200000.vcf.stat

In [ ]:
%%bash
head -n 35 rice-CHR6.200000.vcf.stat

### <span style="color: #4CACBC;">Generating statistics from a VCF to determine how to set filters on it<a class="anchor" id="vcffilters"></a></span> `vcftools`

We will generate more statistics from a VCF using vcftools (LINK MANUAL), a very useful and fast program for handling vcf files 
to easily calculate these statistics in order to better define filters we will apply and to get an idea of how to set such filtering thresholds. 

The main information we will consider are:
* Depth: Usually, we filter SNP with a minimum and maximum depth. We use a minimum depth cutoffs to remove false positive calls and to keep higher quality calls too. 
A maximum cut off allow to remove regions with very, very high read depths such as repetitive regions.
* Quality Genotype quality : With this filter, we should not trust any genotype with a Phred score below 20 which suggests a less than 99% accuracy.
* Minor allele frequency MAF can cause big problems with SNP calls - and also inflate statistical estimates downstream. Ideally you want an idea of the distribution of your 
allelic frequencies but 0.05 to 0.10 is a reasonable cut-off. You should keep in mind however that some analyses, particularly demographic inference can be biased by MAF thresholds.
* Missing data How much missing data are you willing to tolerate? It will depend on the study but typically any site with >25% missing data should be dropped.
* biallelic, heterozygosity...

In this training, we will just display quality and depth distribution... but you should do on each value filterd.

#### <span style="color: #4CACBC;">Mean depth per individual and per site<a class="anchor" id="depthvcf"></a></span> `--depth, --site-mean-depth`

* run vcftool with the correct options
* check that the files have been created and display the first lines

In [ ]:
%%bash
vcftools --gzvcf rice-CHR6.200000.vcf.gz --depth --out depthi
vcftools --gzvcf rice-CHR6.200000.vcf.gz --site-mean-depth --out depths

In [ ]:
%%bash
ls -lrt
head *depth*

#### <span style="color: #4CACBC;">Extracting quality per site<a class="anchor" id="depthvcf"></a></span>  `--site-quality`

* run vcftool with the correct options
* check that the files have been created and display the first lines

In [ ]:
%%bash
vcftools --gzvcf rice-CHR6.200000.vcf.gz  --site-quality --out qual

In [ ]:
%%bash
ls -lrt
head *qual


__Calculate allele frequency__

* --freq2 : outputs the frequencies without information about the alleles
* --freq would return their identity. 
* --max-alleles 2 to exclude sites that have more than two alleles.

In [ ]:
%%bash
vcftools --gzvcf rice-CHR6.200000.vcf.gz --freq --out AF --max-alleles 2
vcftools --gzvcf rice-CHR6.200000.vcf.gz --freq2 --out AF2 --max-alleles 2
ls -lrt
head *.frq

### <span style="color: #4CACBC;">Generating density plot QUAL & DEPTH<a class="anchor" id="vcfplot"></a></span> 

#### <span style="color: #4CACBC;">Plotting quality per site<a class="anchor" id="qualplot"></a></span> 

In [ ]:
qual_file="qual.lqual"
df_qual = pd.read_csv(qual_file, index_col=False, sep="\t")
print(df_qual)
df_qual['QUAL'].describe()

In [ ]:
# Plot with seaborn
import matplotlib.pyplot as plt
import seaborn as sns

sns.kdeplot(x="QUAL", data=df_qual)

In [ ]:
# Plot with seaborn
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize = (15,8))
ax=sns.kdeplot(x="QUAL", data=df_qual[df_qual.QUAL<10000])
ax.set_title("PUT A TITLE")
ax.set_xlabel("PUT A X-AXIS LABEL")
ax.set_ylabel("PUT A Y-AXIS LABEL")

#### <span style="color: #4CACBC;">Plotting Mean depth per site<a class="anchor" id="depthplot"></a></span> 

In [ ]:
depth_file="depths.ldepth.mean"
df_depth = pd.read_csv(depth_file, index_col=False, sep="\t")
print(df_depth)
df_depth.describe()

In [ ]:
# Plot with seaborn
import matplotlib.pyplot as plt
import seaborn as sns

ax=sns.kdeplot(x="MEAN_DEPTH", data=df_depth)
ax.set_xlim(1, 70)
ax.set_title("PUT A TITLE")
ax.set_xlabel("PUT A X-AXIS LABEL")
ax.set_ylabel("PUT A Y-AXIS LABEL")

In [ ]:
# Plot with seaborn
import matplotlib.pyplot as plt
import seaborn as sns

ax=sns.kdeplot(x="MEAN_DEPTH", data=df_depth[df_depth.MEAN_DEPTH<100])
ax.set_xlim(1, 70)
ax.set_title("PUT A TITLE")
ax.set_xlabel("PUT A X-AXIS LABEL")
ax.set_ylabel("PUT A Y-AXIS LABEL")

### <span style="color: #4CACBC;">III - FILTERING VCF <a class="anchor" id="vcffiltering"></a></span> 


#### Which filters ?

Here, we will apply on the vcf the following filters :

* QUAL > 3000
* DP > 10 and DP < 20000
* Less than 3 SNPs into a window of 10pb

The threshodl of each filter depends on the SNP analysis (sample number, sequencing depth).

In a second step, according the following analysys (eg: population genomics), usually, we will apply other filters such as :
* removing missing data
* keeping only biallellic
* heterozygosity
    
#### Select only the SNPs (and remove the INDELs) - `gatk variantFiltration`
* Filter vcf to keep only SNPs
* Check that the new vcf has been created
* Get the number of polymorphisms in the new vcf file 

In [ ]:
%%bash
gatk SelectVariants --java-options "-Xmx8G -Xms8G" -R genome-Osativa-MSU7.fa -V rice-CHR6.200000.vcf.gz -select-type SNP -O rice-CHR6.200000.onlySNP.vcf

In [ ]:
%%bash
ls -lrth
bcftools stats rice-CHR6.200000.onlySNP.vcf | head -n35

#### Compress the vcf and generate the index of the compressed vcf - `tabix -p vcf vcf_file`

In [ ]:
%%bash
bgzip rice-CHR6.200000.onlySNP.vcf
ls -lrth

In [ ]:
%%bash
tabix -f -p vcf rice-CHR6.200000.onlySNP.vcf.gz
ls -lrth

#### Applying filters on QUAL, DEPTH and CLUSTER SNPsSelect only the SNPs (and remove the INDELs) - `gatk variantFiltration`

In [ ]:
%%bash
gatk VariantFiltration --java-options "-Xmx12G -Xms12G" -R genome-Osativa-MSU7.fa -V rice-CHR6.200000.onlySNP.vcf.gz --filter-expression "QUAL<200" --filter-name "LOW_QUAL" --filter-expression "DP<10" --filter-name "LOW_DP"     --cluster-size 3 --cluster-window-size 10     --filter-expression "DP>20000" --filter-name "HIGH-DP" -O rice-CHR6.200000.filteredSNP.vcf

In [ ]:
ls -lrtht

In [ ]:
%%bash
bgzip rice-CHR6.200000.filteredSNP.vcf
tabix -p vcf rice-CHR6.200000.filteredSNP.vcf.gz
ls -lrtht

In [ ]:
%%bash
bcftools stats rice-CHR6.200000.filteredSNP.vcf.gz | head -n 35

#### _Commands to apply other filters_

##### remove missing data if more than 5% of missing data at a site (12 samples over 228 samples) `vcftools`

`vcftools --vcf input.vcf --max-missing-count 12 --remove-filtered-all --recode --recode-INFO-all  --out ouput.onlySNP.12na`

##### keep SNPs with only 90% homzygous (222 samples)

`cat input.vcf | java -jar SnpSift.jar filter " (countHom( )> 222) " > output.onlySNP.12na.recode.222homoz.vcf`

##### keep only biallelic sites
`vcftools --vcf input.vcf --min-alleles 2 --max-alleles 2 --remove-filtered-all --recode --recode-INFO-all --out output.onlySNP.filteredPASS.na.225.Homoz.biallelic`

### <span style="color: #4CACBC;">IV - VCF ANNOTATION <a class="anchor" id="vcffiltering"></a></span> 

java -jar snpEff.jar -v -o statschr6.onlySNP.html rice6.1 input.vcf > output.snpeff.vcf

In [ ]:
df_csv_stat["MAF"]=df_csv_stat[['ALL1', 'ALL2']].min(axis=1)


In [ ]:
print(df_csv_stat[df_csv_stat.ALL1>0.70])

In [ ]:
# Plot with seaborn
import matplotlib.pyplot as plt
import seaborn as sns

sns.kdeplot(x="MAF", data=df_csv_stat)

In [ ]:
cd $vcf_dir

In [ ]:
%%bash
wget --no-check-certificat https://itrop.ird.fr/sv-training/out.vcf.gz

In [ ]:
ls

In [ ]:
%%bash
zgrep -vc "^#" out.vcf.gz | head

In [ ]:
%%bash
REF="/home/jovyan/work/DATA/Clone10/referenceCorrect.fasta"
cd /home/jovyan/work/MAPPING-ILL/VCF/
#tail -n 50 /home/jovyan/work/MAPPING-ILL/VCF/Clone1.g.vcf
gatk CombineGVCFs -R $REF --variant Clone1.g.vcf --variant Clone2.g.vcf  -O final.vcf

In [ ]:
%%bash

REF="/home/jovyan/work/DATA/Clone10/referenceCorrect.fasta"
cd /home/jovyan/work/MAPPING-ILL/VCF/
ls -lrt
head -n 1000 final.vcf | tail

gatk --java-options "-Xmx4g" GenotypeGVCFs -R $REF -V final.vcf -O final.genotype.vcf

In [ ]:
%%bash

REF="/home/jovyan/work/DATA/Clone10/referenceCorrect.fasta"
cd /home/jovyan/work/MAPPING-ILL/VCF/
ls -lrt
head -n 1000 final.genotype.vcf | tail


In [ ]:
%%bash

REF="/home/jovyan/work/DATA/Clone10/referenceCorrect.fasta"
VCF=home/jovyan/work/MAPPING-ILL/VCF/final.genotype.vcf
cd /home/jovyan/work/MAPPING-ILL/VCF/f
grep  -vc "^#" /

bcftools stats /home/jovyan/work/MAPPING-ILL/VCF/final.genotype.vcf | head -n 50

In [ ]:
eles, --freq would return their identity. We need to add max-alleles 2 to exclude sites that have more than two alleles.

vcftools --gzvcf $SUBSET_VCF --freq2 --out $OUT --max-alleles 2

In [ ]:
grep -vc "^#" out.vcf.gz